In [16]:
# !pip install sqlalchemy
# !pip install sqlalchemy-trino

from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.errors import PySparkException
from pyspark.sql.types import StructType, StructField, StringType, TimestampType , LongType
import socket
from sqlalchemy import create_engine, text

import os

# from sqlalchemy import *
# from sqlalchemy.engine import create_engine
# from sqlalchemy.schema import *

os.environ["AWS_REGION"] = "us-west-2"

conf = SparkConf() \
    .setAppName("spark demo") \
    .setMaster("spark://172.30.128.211:7077") \
    .set("spark.driver.host", socket.gethostbyname(socket.gethostname()))\
    .set("spark.driver.port", "10027") 


spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()


In [17]:


# Verify the Spark session configuration
for item in spark.sparkContext.getConf().getAll():
    print(item)


('spark.sql.shuffle.partitions', '40')
('spark.hadoop.fs.s3a.access.key', 'minioAdmin')
('spark.eventLog.enabled', 'true')
('spark.app.submitTime', '1732570968360')
('spark.hadoop.fs.s3a.path.style.access', 'true')
('spark.app.id', 'app-20241125215025-0010')
('spark.master', 'spark://172.30.128.211:7077')
('spark.sql.catalog.apm.type', 'hive')
('spark.app.startTime', '1732571425495')
('spark.sql.catalog.apm.s3.path-style-access', 'true')
('spark.driver.host', '10.128.5.68')
('spark.hadoop.hive.metastore.schema.verification.record.version', 'false')
('spark.hadoop.fs.s3a.secret.key', 'minio1234')
('spark.serializer.objectStreamReset', '100')
('spark.submit.deployMode', 'client')
('spark.sql.catalog.apm', 'org.apache.iceberg.spark.SparkCatalog')
('spark.sql.catalog.apm.uri', 'thrift://hive-metastore:9083')
('spark.eventLog.dir', 's3a://spark-events/logs/')
('spark.sql.catalog.apm.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
('spark.executor.memory', '2g')
('spark.hadoop.fs.s3a.endpoin

In [18]:
spark

In [19]:
emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","Female","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"],
    ["022","108","Jey Paulraj2","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [20]:


try:
    # Create Iceberg table using the specified schema and catalog
    print("Iceberg databae created  1------ ")
    print("current database" , spark.catalog.currentDatabase())
    print("list of database" , spark.catalog.listDatabases())
    spark.sql(""" CREATE DATABASE IF NOT EXISTS observability  """)
    
    # spark.sql("create database observability location '/user/data_transform/iceberg_test'")
   
    print("Iceberg databae created  ------ ")
   

    spark.sql("""
        CREATE TABLE IF NOT EXISTS observability.employee (
            employee_id string,
            department_id string,
            name string,
            age string,
            gender string,
            salary string,
            hire_date string            
        )  USING iceberg  """)
    
    print("Iceberg table trace  created ------ ")
   
    print("Iceberg table 'apm.trace' created successfully.")
    
except Exception as e:
    print("Error creating Iceberg table:", str(e))

 


Iceberg databae created  1------ 
current database 


list of database [Database(name='default', catalog='apm', description='Default Hive database', locationUri='s3a://pcaf/data'), Database(name='demo', catalog='apm', description=None, locationUri='s3a://pcaf/datamesh/observability/demo'), Database(name='infra_node', catalog='apm', description=None, locationUri='s3a://pcaf/data/infra_node.db'), Database(name='infra_pod', catalog='apm', description=None, locationUri='s3a://pcaf/data/infra_pod.db'), Database(name='observability', catalog='apm', description=None, locationUri='s3a://datamesh/observability/observability.db'), Database(name='pcaf', catalog='apm', description=None, locationUri='s3a://pcaf/'), Database(name='product', catalog='apm', description=None, locationUri='s3a://pcaf/datamesh/prodcut'), Database(name='sustainability_node', catalog='apm', description=None, locationUri='s3a://pcaf/data/sustainability_node.db'), Database(name='sustainability_pod', catalog='apm', description=None, locationUri='s3a://pcaf/data/sustainability_po

In [21]:
emp = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [22]:
# Show data (ACTION)

emp.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [23]:
emp.writeTo("observability.employee").append()
# emp.write.format("iceberg").mode("append").save("observability.db.employee")

24/11/25 21:50:36 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


In [24]:
df = spark.table("observability.employee").show(n=20)

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [25]:
engine = create_engine('trino://admin@trino:8080')

In [26]:
engine.connect

<bound method Engine.connect of Engine(trino://admin@trino:8080)>

In [27]:
with engine.connect() as connection:
    result = connection.execute(text('show catalogs'))
    for row in result:
        print(row)

('hive',)
('iceberg',)
('system',)
('tpcds',)
('tpch',)
('unity',)


In [28]:
# Show available tables
tables_show_sql = f"""
show tables from iceberg.observability
"""
with engine.connect() as connection:
    result = connection.execute(text(tables_show_sql))
    for row in result:
        print(row)

('employee',)


In [29]:
# Query table to verify insertion was successful
select_query = f"""
select * from iceberg.observability.employee
"""
print(select_query)
with engine.connect() as connection:
    result = connection.execute(text(select_query)).fetchmany(200)
    for row in result:
        print(row)


select * from iceberg.observability.employee

('019', '103', 'Steven Chen', '36', 'Male', '62000', '2015-08-01')
('020', '102', 'Grace Kim', '32', 'Female', '53000', '2018-11-01')
('022', '108', 'Jey Paulraj2', '32', 'Female', '53000', '2018-11-01')
('019', '103', 'Steven Chen', '36', 'Male', '62000', '2015-08-01')
('020', '102', 'Grace Kim', '32', 'Female', '53000', '2018-11-01')
('022', '108', 'Jey Paulraj2', '32', 'Female', '53000', '2018-11-01')
('017', '105', 'George Wang', '34', 'Male', '57000', '2016-03-15')
('009', '103', 'Tom Tan', '33', 'Male', '58000', '2016-06-01')
('010', '104', 'Lisa Lee', '27', 'Female', '47000', '2018-08-01')
('018', '104', 'Nancy Liu', '29', 'Female', '50000', '2017-06-01')
('001', '101', 'John Doe', '30', 'Male', '50000', '2015-01-01')
('017', '105', 'George Wang', '34', 'Male', '57000', '2016-03-15')
('002', '101', 'Jane Smith', '25', 'Female', '45000', '2016-02-15')
('003', '102', 'Bob Brown', '35', 'Male', '55000', '2014-05-01')
('018', '104', 'Na

In [30]:
spark.stop()